# MetadataGenerator (Hive Queries)

This notebook demonstrates how MetadataGeneratorHiveQuery generate metadata from SQL files.
The following use cases are covered:
* Simmple SQL
* Case statement
* Multiple case statements
* Temporary tables
* Sub-queries
* Various Columns with functions

Also, this notebook demonstrate how it reformates SQL query.

In [1]:
from MetadataGeneratorHiveQuery import generate_metadata_from_hive_query

In [2]:
idx_query = 0

# Use Case 1 (simple)

This is the simplest SQL query. The query pull columns from source table without any transformation.

In [38]:
file_path = '../data/sample.sql'

In [39]:
result = generate_metadata_from_hive_query(
    file_path = file_path,
    idx_query = idx_query
)

In [14]:
print(result[idx_query]['query'])

CREATE TABLE schema.table AS
SELECT col1,
       col2
FROM schema_source.table_source;


In [15]:
for item in result[idx_query]['metadata_query']:
    print(item)

{'token': 'create', 'type': 'table', 'value': 'schema.table', 'metadta': {'schema_name': 'schema', 'table_name': 'table', 'table_alias': None}}
{'token': 'select', 'type': 'column', 'value': 'col1', 'metadata': {'column_name': 'col1'}}
{'token': 'select', 'type': 'column', 'value': 'col2', 'metadata': {'column_name': 'col2'}}
{'token': 'FROM', 'type': 'table', 'value': 'schema_source.table_source', 'metadata': {'schema_name': 'schema_source', 'table_name': 'table_source', 'table_alias': None}}


# Use Case 2 (CASE)

This query involves single non-nested Case statement.

In [41]:
file_path = '../data/sample_case.sql'

In [42]:
result = generate_metadata_from_hive_query(
    file_path = file_path,
    idx_query = idx_query
)

In [43]:
print(result[idx_query]['query'])

CREATE TABLE schema.table AS
SELECT CASE
           WHEN a.col2 = 15
                AND a.col3 < a.col1 THEN 1
           ELSE NULL
       END AS col2_ind
FROM schema_name.table_name_case AS a;


In [44]:
for item in result[idx_query]['metadata_query']:
    print(item)

{'token': 'create', 'type': 'table', 'value': 'schema.table', 'metadta': {'schema_name': 'schema', 'table_name': 'table', 'table_alias': None}}
{'token': 'select', 'type': 'case', 'is_nested': False, 'metadta': [{'when': 'a.col2 = 15'}, {'when': 'a.col3 < a.col1'}, {'then': '1'}, {'else': 'Null'}, {'column_name': 'col2_ind'}]}
{'token': 'FROM', 'type': 'table', 'value': 'schema_name.table_name_case as a', 'metadata': {'schema_name': 'schema_name', 'table_name': 'table_name_case', 'table_alias': 'a'}}


# Use Case 3 (multiple CASES)

This query involves a nested Case statement.

In [45]:
file_path = '../data/sample_cases.sql'

In [46]:
result = generate_metadata_from_hive_query(
    file_path = file_path,
    idx_query = idx_query
)

In [47]:
print(result[idx_query]['query'])

CREATE TABLE schema.table AS
SELECT a.col1,
       CASE
           WHEN a.col2 = 15 THEN 1
           ELSE CASE
                    WHEN a.col2 < 0 THEN 2
                    ELSE CASE
                             WHEN a.col2 > 15
                                  AND a.col1 = a.col3 THEN 3
                             ELSE 0
                         END
                END
       END AS col2_ind
FROM schema_name.table_name_case AS a;


In [48]:
for item in result[idx_query]['metadata_query']:
    print(item)

{'token': 'create', 'type': 'table', 'value': 'schema.table', 'metadta': {'schema_name': 'schema', 'table_name': 'table', 'table_alias': None}}
{'token': 'select', 'type': 'column', 'value': 'a.col1', 'metadata': {'column_name': 'col1', 'table_alias': 'a'}}
{'token': 'select', 'type': 'case', 'is_nested': True, 'metadta': [{'when': 'a.col2 = 15'}, {'then': '1'}, {'else': 'nested_case'}, {'column_name': 'col2_ind'}, [{'when': 'a.col2 < 0'}, {'then': '2'}, {'else': 'nested_case'}], [{'when': 'a.col2 > 15'}, {'when': 'a.col1 = a.col3'}, {'then': '3'}, {'else': '0'}]]}
{'token': 'FROM', 'type': 'table', 'value': 'schema_name.table_name_case as a', 'metadata': {'schema_name': 'schema_name', 'table_name': 'table_name_case', 'table_alias': 'a'}}


# Use Case 4 (Temporary Tables)

This involves multiple temporary tables.

In [49]:
file_path = '../data/sample_tmp_tables.sql'

In [50]:
result = generate_metadata_from_hive_query(
    file_path = file_path,
    idx_query = idx_query
)

In [51]:
print(result[idx_query]['query'])

CREATE TABLE schema.table AS WITH tmp_table_0 AS
  (SELECT a.col1,
          a.col2,
          a.col3,
          a.col4
   FROM schema_name_2.table_name_source0 AS a),
                                  tmp_table_1 AS
  (SELECT b.col1,
          b.col4,
          b.col5,
          b.col6
   FROM schema_name_2.table_name_source1 AS b)
SELECT A.col1,
       A.col2,
       A.col3,
       B.col4,
       CASE
           WHEN b.col5 = a.col4 THEN 1
           ELSE 0
       END AS col5,
       COALESCE(b.col6,
                0) AS col6_fill_na
FROM tmp_table_0 AS A,
     tmp_table_1 AS B
WHERE a.col1 = b.col1;


In [52]:
for item in result[idx_query]['metadata_query']:
    print(item)

{'token': 'create', 'type': 'table', 'value': 'schema.table', 'metadta': {'schema_name': 'schema', 'table_name': 'table', 'table_alias': None}}
{'token': 'with', 'type': 'temporary table', 'table_alias': None, 'value': [{'token': 'select', 'type': 'column', 'value': 'b.col1', 'metadata': {'column_name': 'col1', 'table_alias': 'b'}}, {'token': 'select', 'type': 'column', 'value': 'b.col4', 'metadata': {'column_name': 'col4', 'table_alias': 'b'}}, {'token': 'select', 'type': 'column', 'value': 'b.col5', 'metadata': {'column_name': 'col5', 'table_alias': 'b'}}, {'token': 'select', 'type': 'column', 'value': 'b.col6', 'metadata': {'column_name': 'col6', 'table_alias': 'b'}}, {'token': 'FROM', 'type': 'table', 'value': 'schema_name_2.table_name_source1 AS b', 'metadata': {'schema_name': 'schema_name_2', 'table_name': 'table_name_source1', 'table_alias': 'b'}}]}
{'token': 'with', 'type': 'temporary table', 'table_alias': None, 'value': [{'token': 'select', 'type': 'column', 'value': 'b.col1'

# Use Case 5 (Various Columns)

This query includes various functions.

In [28]:
file_path = '../data/sample_various_columns.sql'

In [29]:
result = generate_metadata_from_hive_query(
    file_path = file_path,
    idx_query = idx_query
)

In [30]:
print(result[idx_query]['query'])

CREATE TABLE schema.table AS
SELECT a.col1,
       COALESCE(a.col2,
                0) AS col2_coalesce,
       NVL(a.col3,
           0) AS col3_nvl,
       CASE
           WHEN a.col3 == b.col3 THEN 1
           ELSE 0
       END AS col3_comaprison_ind,
       b.col4 AS col4_renamed,
       count(b.col5) AS col5_sum,
       1 AS col6,
       b.col7 - b.col6 AS col7,
       1.1 AS col8,
       'A' AS col9,
       cast(b.col10 AS bigint) AS col10_bigint,
       col11
FROM schema_name.table_name_various AS a
JOIN schema_name.table_name_various AS b ON a.col1 == b.col2
AND a.col2 > b.col2;


In [31]:
for item in result[idx_query]['metadata_query']:
    print(item)

{'token': 'create', 'type': 'table', 'value': 'schema.table', 'metadta': {'schema_name': 'schema', 'table_name': 'table', 'table_alias': None}}
{'token': 'select', 'type': 'column', 'value': 'a.col1', 'metadata': {'column_name': 'col1', 'table_alias': 'a'}}
{'token': 'select', 'type': 'column', 'value': 'COALESCE(a.col2, 0) as col2_coalesce', 'metadata': {'column_name': 'col2_coalesce', 'is_function': True}}
{'token': 'select', 'type': 'column', 'value': 'NVL(a.col3, 0) as col3_nvl', 'metadata': {'column_name': 'col3_nvl', 'is_function': True}}
{'token': 'select', 'type': 'case', 'is_nested': False, 'metadta': [{'when': 'a.col3 == b.col3'}, {'then': '1'}, {'else': '0'}, {'column_name': 'col3_comaprison_ind'}]}
{'token': 'select', 'type': 'column', 'value': 'b.col4 AS col4_renamed', 'metadata': {'column_name': 'col4_renamed', 'column_name_before_rename': 'col4', 'table_alias': 'b'}}
{'token': 'select', 'type': 'column', 'value': 'count(b.col5) AS col5_sum', 'metadata': {'column_name': '

# Use Case 6 (Sub-queries)

This includes a nested sub-query.

In [53]:
file_path = '../data/sample_subquery.sql'

In [54]:
result = generate_metadata_from_hive_query(
    file_path = file_path,
    idx_query = idx_query
)

In [55]:
print(result[idx_query]['query'])

CREATE TABLE schema.table AS
SELECT a.col1,
       a.col2,
       b.col4
FROM
  (SELECT c.col1,
          c.col2
   FROM
     (SELECT d.col1,
             d.col2
      FROM schema_name_2.table_name_11_0 AS d) AS c) AS a
JOIN schema_name.table_name_subquery AS b ON a.col1 = b.col1;


In [56]:
for item in result[idx_query]['metadata_query']:
    print(item)

{'token': 'create', 'type': 'table', 'value': 'schema.table', 'metadta': {'schema_name': 'schema', 'table_name': 'table', 'table_alias': None}}
{'token': 'select', 'type': 'column', 'value': 'a.col1', 'metadata': {'column_name': 'col1', 'table_alias': 'a'}}
{'token': 'select', 'type': 'column', 'value': 'a.col2', 'metadata': {'column_name': 'col2', 'table_alias': 'a'}}
{'token': 'select', 'type': 'column', 'value': 'b.col4', 'metadata': {'column_name': 'col4', 'table_alias': 'b'}}
{'token': 'FROM', 'type': 'subquery', 'value': [{'token': 'select', 'type': 'column', 'value': 'c.col1', 'metadata': {'column_name': 'col1', 'table_alias': 'c'}}, {'token': 'select', 'type': 'column', 'value': 'c.col2', 'metadata': {'column_name': 'col2', 'table_alias': 'c'}}, {'token': 'FROM', 'type': 'subquery', 'value': [{'token': 'select', 'type': 'column', 'value': 'd.col1', 'metadata': {'column_name': 'col1', 'table_alias': 'd'}}, {'token': 'select', 'type': 'column', 'value': 'd.col2', 'metadata': {'co

# Use Case 7 (Having)

This query involves Having statement.

In [3]:
file_path = '../data/sample_having.sql'

In [4]:
result = generate_metadata_from_hive_query(
    file_path = file_path,
    idx_query = idx_query
)

In [5]:
print(result[idx_query]['query'])

CREATE TABLE schema.table AS
SELECT a.col1,
       a.col2,
       COUNT(DISTINCT a.col3) AS col3_sum_distinct,
       COUNT(a.col3) AS col3_sum
FROM schmea_name.table_name AS a
WHERE a.col3 IN (1,
                 2)
GROUP BY a.col1,
         a.col2
HAVING COUNT(DISTINCT a.col3) > 1
OR a.col3 == -10


In [6]:
for item in result[idx_query]['metadata_query']:
    print(item)

{'token': 'create', 'type': 'table', 'value': 'schema.table', 'metadta': {'schema_name': 'schema', 'table_name': 'table', 'table_alias': None}}
{'token': 'select', 'type': 'column', 'value': 'a.col1', 'metadata': {'column_name': 'col1', 'table_alias': 'a'}}
{'token': 'select', 'type': 'column', 'value': 'a.col2', 'metadata': {'column_name': 'col2', 'table_alias': 'a'}}
{'token': 'select', 'type': 'column', 'value': 'COUNT(DISTINCT a.col3) AS col3_sum_distinct', 'metadata': {'column_name': 'col3_sum_distinct', 'is_function': True}}
{'token': 'select', 'type': 'column', 'value': 'COUNT(a.col3) AS col3_sum', 'metadata': {'column_name': 'col3_sum', 'is_function': True}}
{'token': 'FROM', 'type': 'table', 'value': 'schmea_name.table_name AS a', 'metadata': {'schema_name': 'schmea_name', 'table_name': 'table_name', 'table_alias': 'a'}}
{'token': 'GROUP BY', 'type': 'column', 'value': 'a.col1', 'metadata': {'column_name': 'col1', 'table_alias': 'a'}}
{'token': 'GROUP BY', 'type': 'column', 'v